In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import numpy as np
import os
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from sklearn.decomposition import PCA
import cv2


In [3]:
directory = r'./drive/MyDrive/Brain tumor dataset RM/split_brain_tumor_dataset'
labels = os.listdir(directory)
labels

['test', 'train']

In [4]:
input_dir = './drive/MyDrive/Brain tumor dataset RM/split_brain_tumor_dataset/train'
output_dir = './drive/MyDrive/Brain tumor dataset RM/augmented'
labels = ['yes', 'no']

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

datagen = ImageDataGenerator(
    rotation_range=5,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.05,
    horizontal_flip=True,
    # vertical_flip=True,
    fill_mode='nearest'
)

def augment_images(label):
    label_dir = os.path.join(input_dir, label)
    save_to_dir = os.path.join(output_dir, label)
    if not os.path.exists(save_to_dir):
        os.makedirs(save_to_dir)

    images = os.listdir(label_dir)
    total_images = len(images)
    augment_factor = 600 // total_images

    for image_name in images:
        image_path = os.path.join(label_dir, image_name)
        img = load_img(image_path)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)

        i = 0
        for batch in datagen.flow(x, batch_size=1, save_to_dir=save_to_dir, save_prefix=label, save_format='jpeg'):
            i += 1
            if i > augment_factor:
                break
for label in labels:
    augment_images(label)

In [6]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img

input_dir = './drive/MyDrive/Brain tumor dataset RM/augmented'
output_dir = './drive/MyDrive/Brain tumor dataset RM/otsu_augmented'
labels = ['yes', 'no']

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def apply_otsu_thresholding(label):
    label_dir = os.path.join(input_dir, label)
    save_to_dir = os.path.join(output_dir, label)
    if not os.path.exists(save_to_dir):
        os.makedirs(save_to_dir)

    images = os.listdir(label_dir)

    for image_name in images:
        image_path = os.path.join(label_dir, image_name)
        img = load_img(image_path, color_mode='grayscale')
        img_array = img_to_array(img)
        img_array = img_array.astype('uint8')

        # Remove the extra channel dimension
        img_array = img_array.squeeze()

        # Apply Otsu thresholding
        _, otsu_thresholded = cv2.threshold(img_array, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Add the channel dimension back for saving
        otsu_thresholded = np.expand_dims(otsu_thresholded, axis=-1)

        # Convert back to image format
        otsu_img = array_to_img(otsu_thresholded)

        # Save the image
        otsu_img.save(os.path.join(save_to_dir, image_name))

for label in labels:
    apply_otsu_thresholding(label)


## Extract Otsu Test

In [7]:
import os
import cv2
import numpy as np
from keras.preprocessing.image import img_to_array, load_img

input_dir = './drive/MyDrive/Brain tumor dataset RM/split_brain_tumor_dataset/test'
output_dir = './otsu_test'
labels = ['yes', 'no']

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def apply_otsu_thresholding(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (1, 1), 0)
    _, otsu_image = cv2.threshold(blurred_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return otsu_image

def process_images(label):
    label_dir = os.path.join(input_dir, label)
    save_to_dir = os.path.join(output_dir, label)
    if not os.path.exists(save_to_dir):
        os.makedirs(save_to_dir)

    images = os.listdir(label_dir)
    for image_name in images:
        image_path = os.path.join(label_dir, image_name)
        img = cv2.imread(image_path)
        otsu_img = apply_otsu_thresholding(img)
        save_path = os.path.join(save_to_dir, image_name)
        cv2.imwrite(save_path, otsu_img)

for label in labels:
    process_images(label)


In [ ]:
augmented_dir = './otsu_test'
labels = ['yes', 'no']

base_model = VGG16(weights='imagenet', include_top=True)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

def extract_features(directory, label):
    features = []
    image_labels = []
    images = os.listdir(directory)

    for image_name in images:
        image_path = os.path.join(directory, image_name)
        img = load_img(image_path, target_size=(224, 224))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        feature = model.predict(x)
        feature = feature.flatten()

        features.append(feature)
        image_labels.append(label)

    return features, image_labels

all_features = []
all_labels = []

for label in labels:
    dir_path = os.path.join(augmented_dir, label)
    features, image_labels = extract_features(dir_path, label)
    all_features.extend(features)
    all_labels.extend(image_labels)

features_df = pd.DataFrame(all_features)
features_df['label'] = all_labels

features_df.to_csv('./drive/MyDrive/Brain tumor dataset RM/result_csv/test_otsu_data.csv', index=False)

print("Feature extraction and CSV saving completed.")


## Extract Not Otsu Test

In [ ]:
augmented_dir = './drive/MyDrive/Brain tumor dataset RM/split_brain_tumor_dataset/test'
labels = ['yes', 'no']

base_model = VGG16(weights='imagenet', include_top=True)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

def extract_features(directory, label):
    features = []
    image_labels = []
    images = os.listdir(directory)

    for image_name in images:
        image_path = os.path.join(directory, image_name)
        img = load_img(image_path, target_size=(224, 224))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        feature = model.predict(x)
        feature = feature.flatten()

        features.append(feature)
        image_labels.append(label)

    return features, image_labels

all_features = []
all_labels = []

for label in labels:
    dir_path = os.path.join(augmented_dir, label)
    features, image_labels = extract_features(dir_path, label)
    all_features.extend(features)
    all_labels.extend(image_labels)

features_df = pd.DataFrame(all_features)
features_df['label'] = all_labels

features_df.to_csv('./drive/MyDrive/Brain tumor dataset RM/result_csv/test_not_otsu_data.csv', index=False)

print("Feature extraction and CSV saving completed.")


## Extract Otsu Augmented

In [14]:
augmented_dir = './drive/MyDrive/Brain tumor dataset RM/otsu_augmented'
labels = ['yes', 'no']

base_model = VGG16(weights='imagenet', include_top=True)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

def extract_features(directory, label):
    features = []
    image_labels = []
    images = os.listdir(directory)

    for image_name in images:
        image_path = os.path.join(directory, image_name)
        img = load_img(image_path, target_size=(224, 224))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        feature = model.predict(x)
        feature = feature.flatten()

        features.append(feature)
        image_labels.append(label)

    return features, image_labels

all_features = []
all_labels = []

for label in labels:
    dir_path = os.path.join(augmented_dir, label)
    features, image_labels = extract_features(dir_path, label)
    all_features.extend(features)
    all_labels.extend(image_labels)

features_df = pd.DataFrame(all_features)
features_df['label'] = all_labels

features_df.to_csv('./drive/MyDrive/Brain tumor dataset RM/result_csv/train_otsu_data.csv', index=False)

print("Feature extraction and CSV saving completed.")


1/1 [==============================] - 1s 529ms/step
Feature extraction and CSV saving completed.


## Extract Not Otsu Augmented

In [15]:
augmented_dir = './drive/MyDrive/Brain tumor dataset RM/augmented'
labels = ['yes', 'no']

base_model = VGG16(weights='imagenet', include_top=True)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

def extract_features(directory, label):
    features = []
    image_labels = []
    images = os.listdir(directory)

    for image_name in images:
        image_path = os.path.join(directory, image_name)
        img = load_img(image_path, target_size=(224, 224))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        feature = model.predict(x)
        feature = feature.flatten()

        features.append(feature)
        image_labels.append(label)

    return features, image_labels

all_features = []
all_labels = []

for label in labels:
    dir_path = os.path.join(augmented_dir, label)
    features, image_labels = extract_features(dir_path, label)
    all_features.extend(features)
    all_labels.extend(image_labels)

features_df = pd.DataFrame(all_features)
features_df['label'] = all_labels

features_df.to_csv('./drive/MyDrive/Brain tumor dataset RM/result_csv/train_not_otsu_data.csv', index=False)

print("Feature extraction and CSV saving completed.")


1/1 [==============================] - 1s 525ms/step
Feature extraction and CSV saving completed.
